In [1]:
# 필요 모듈 임포트
import numpy as np
import pandas as pd
import gzip
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

%matplotlib inline

### user-related (고객 고유 특성 관련 데이터 추가 전처리)
- drop, NaN 등 처리
- user_final.pickle 생성

In [ ]:
# all_info.pickle : 고객고유특성 파생변수 포함 데이터 읽어오기
with gzip.open('../preprocessed/all_info.pickle','rb') as f:
    user = pickle.load(f)

In [3]:
user.shape

(1029390, 331)

In [ ]:
user.columns

In [ ]:
user.head()

In [ ]:
# user related features 전처리
# A별 B 파생변수에서 열명칭 마지막에 '-1' 있는 열 삭제 (A가 결측치여서 의미 찾을 수 없는 열)
drop_cols = []
for s in user.columns:
    if s[-2:]=='-1':
        drop_cols.append(s)
drop_cols

In [7]:
# # 임의판단하에 포함시키려면 다음 변수를 수정하시오
user_all = user.drop(drop_cols,axis=1)
user_all.shape

(1029390, 321)

In [8]:
#데이터 읽어오기 (temp_is_applied: is_applied만 따로 있는 데이터)
with gzip.open('../preprocessed/temp_is_applied.pickle','rb') as f:
    temp_applied = pickle.load(f)

temp_applied

153        NaN
154        NaN
155        NaN
156        1.0
157        1.0
          ... 
1394211    1.0
1394212    NaN
1394213    NaN
1394214    NaN
1394215    1.0
Name: application_id, Length: 1029390, dtype: float64

In [9]:
temp_applied.name = 'is_applied'
temp_applied

153        NaN
154        NaN
155        NaN
156        1.0
157        1.0
          ... 
1394211    1.0
1394212    NaN
1394213    NaN
1394214    NaN
1394215    1.0
Name: is_applied, Length: 1029390, dtype: float64

In [ ]:
# 고객고유특성 테이블과 is_applied 정보 연결하기
user_applied = pd.concat([user_all, temp_applied], axis=1)
user_applied

In [11]:
# is_applied: NaN 값의 경우 0을 뜻하기 때문에 fillna(0)으로 NaN 대체
user_applied['is_applied'] = user_applied['is_applied'].fillna(0)
user_applied['is_applied']

153        0.0
154        0.0
155        0.0
156        1.0
157        1.0
          ... 
1394211    1.0
1394212    0.0
1394213    0.0
1394214    0.0
1394215    1.0
Name: is_applied, Length: 1029390, dtype: float64

In [12]:
user_applied['is_applied'].value_counts()

0.0    651834
1.0    377556
Name: is_applied, dtype: int64

In [ ]:
# 결측치 확인
user_applied.isna().sum()

In [14]:
# 단 하나의 null 값이라도 포함된 행은 모두 제거
user_applied = user_applied.dropna(how='any',axis=0)
user_applied.shape

(723526, 322)

In [15]:
user_applied.columns

Index(['application_id', 'user_id', 'gender', 'insert_time', 'credit_score',
       'yearly_income', 'income_type', 'employment_type', 'houseown_type',
       'desired_amount',
       ...
       'purpose_by_yearly_income_cut_SWITCHLOAN',
       'purpose_by_enter_period_cut_BUSINESS',
       'purpose_by_enter_period_cut_BUYCAR',
       'purpose_by_enter_period_cut_BUYHOUSE',
       'purpose_by_enter_period_cut_ETC',
       'purpose_by_enter_period_cut_HOUSEDEPOSIT',
       'purpose_by_enter_period_cut_INVEST',
       'purpose_by_enter_period_cut_LIVING',
       'purpose_by_enter_period_cut_SWITCHLOAN', 'is_applied'],
      dtype='object', length=322)

In [16]:
# 연속형/범주형 변수 뽑기
user_applied['insert_time']=pd.to_datetime(user_applied['insert_time'])
cat_columns = [c for c, t in zip(user_applied.dtypes.index, user_applied.dtypes) if t == 'O']
num_columns = [c for c    in user_applied.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
# print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['income_type', 'employment_type', 'houseown_type', 'purpose']



/tmp/ipykernel_48821/3213798206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_applied['insert_time']=pd.to_datetime(user_applied['insert_time'])


In [ ]:
# 범주형 변수 Ordinal 인코딩
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
user_applied[cat_columns] = ordinal_encoder.fit_transform(user_applied[cat_columns])
user_applied[cat_columns].head()

In [18]:
# 각 income_type 인코딩 내용을 pickle 파일로 저장
with open('../preprocessed/ordinal_encoder.pickle','wb') as f:
    pickle.dump(ordinal_encoder, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# insert_time 열 대신 월일시분초 열 새롭게 추가
user_applied['insert_time']=pd.to_datetime(user_applied['insert_time'])

user_applied['insert_month']=user_applied['insert_time'].dt.month
user_applied['insert_day']=user_applied['insert_time'].dt.day
user_applied['insert_hour']=user_applied['insert_time'].dt.hour
user_applied['insert_minute']=user_applied['insert_time'].dt.minute
user_applied['insert_second']=user_applied['insert_time'].dt.second

user_applied = user_applied.drop('insert_time', axis=1)
user_applied.head()

In [47]:
# 고객고유특성 처리한 내용 추후 활용 위해 저장
with gzip.open('../preprocessed/user_final.pickle','wb') as f:
    pickle.dump(user_applied, f, pickle.HIGHEST_PROTOCOL)

### loan-product-related (loan 관련 데이터 추가 전처리)
- drop, NaN 등 처리
- loan_final.pickle 생성

In [ ]:
# loan_info: loan_result와 관련된 파생변수 포함한 데이터, 읽어오기
with gzip.open('../preprocessed/loan_info.pickle','rb') as f:
    loan = pickle.load(f)

loan.head()

In [ ]:
loan.info()

In [ ]:
loan.head()

In [ ]:
loan['is_applied'].value_counts()

In [58]:
# 활용하지 않을 열 drop
loan = loan.drop(['loanapply_insert_time', 'bank_id', 'product_id'], axis=1)

In [ ]:
loan.shape

In [ ]:
# 단 하나의 null 값이라도 포함된 행은 모두 제거
loan = loan.dropna(how='any',axis=0)
loan.shape

In [ ]:
# 범주형 변수 Ordinal 인코딩
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
loan[['desired_amount_cut']] = ordinal_encoder.fit_transform(loan[['desired_amount_cut']])
loan[['desired_amount_cut']]

In [ ]:
# 범주형으로 바뀐 내용 확인
loan.head()

In [ ]:
loan['is_applied'].value_counts()

In [66]:
# 처리한 loan 관련 데이터 추후 활용 위해 저장
with gzip.open('../preprocessed/loan_final.pickle','wb') as f:
    pickle.dump(loan, f, pickle.HIGHEST_PROTOCOL)

### log related (log 관련 데이터 추가 전처리)
- drop, NaN 등 처리
- log_final.pickle 생성

In [ ]:
# log_action_clustering: 군집화 위해 필요한 log 관련 파생변수 포함한 데이터, 읽어오기
with gzip.open('../preprocessed/log_action_clustering.pickle','rb') as f:
    log = pickle.load(f)

log.head()

In [ ]:
log.shape

In [ ]:
log.columns

In [ ]:
log.isna().sum()

In [72]:
# timedelta열 결측치는 -999로 대체
timedelta_col = [col for col in log.columns if 'timedelta' in col]
log[timedelta_col]=log[timedelta_col].fillna(-999)

In [73]:
# 행동 5가지 tf-idf 결측치는 -1로 대체 
actions_col = log.columns[-5:]
log[actions_col]=log[actions_col].fillna(-1)

In [ ]:
# 결측치 잘 처리되었는지 확인
log.isna().sum()

In [75]:
# 추후 활용 위해 저장
with gzip.open('../preprocessed/log_final.pickle','wb') as f:
    pickle.dump(log, f, pickle.HIGHEST_PROTOCOL)